# Wound‑Channel Interference Visualizer (2025-10-28T03:50:08)

Toy 2+1D model of two **Ki‑loops** (spin‑½ defects) approaching each other.
We compute an **echo proxy** \(E(x,y,t) = \langle \psi(t), \psi(t-\Delta t)\rangle\) locally via phase‑field overlap.
Outputs: a sequence of frames you can stitch into an animation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Grid
nx, ny = 200, 200
x = np.linspace(-5, 5, nx)
y = np.linspace(-5, 5, ny)
X, Y = np.meshgrid(x, y, indexing="xy")

def defect_phase(x0, y0, winding=+1):
    # Angular field with core smoothing
    angle = np.arctan2(Y - y0, X - x0)
    r2 = (X - x0)**2 + (Y - y0)**2
    core = np.exp(-r2/0.2)
    return winding*angle, core

def echo_proxy(phi_now, phi_prev, dt=0.05):
    # Local phase echo: cos of phase change weighted by core overlap
    dphi = phi_now - phi_prev
    return np.cos(dphi)

# Trajectories
T = 60
xL0, yL0 = -3.0, 0.0
xR0, yR0 = +3.0, 0.0

frames = []
phi_prev = None
for t in range(T):
    # approach each other
    xL = xL0 + 0.08*t
    xR = xR0 - 0.08*t
    
    phiL, coreL = defect_phase(xL, yL0, +1)
    phiR, coreR = defect_phase(xR, yR0, -1)
    phi = np.unwrap(phiL + phiR)
    if phi_prev is None:
        phi_prev = phi.copy()
    E = echo_proxy(phi, phi_prev)
    phi_prev = phi.copy()
    
    # Constructive vs Destructive regions
    fig, ax = plt.subplots(figsize=(5,4))
    im = ax.imshow(E, extent=[x.min(), x.max(), y.min(), y.max()], origin="lower", vmin=-1, vmax=1)
    ax.set_title(f"Echo proxy E(x,y,t) at t={t}")
    ax.set_xlabel("x"); ax.set_ylabel("y")
    fig.colorbar(im, ax=ax, label="⟨ψ(t), ψ(t-Δt)⟩ proxy")
    out = f"wound_echo_{t:03d}.png"
    plt.tight_layout()
    plt.savefig(out, dpi=120)
    plt.close(fig)

frames[:3] if frames else ["wound_echo_000.png","wound_echo_001.png","wound_echo_002.png"]